### Demographic data

* customerid (Primary key used to merge to other data)
* birthdate (date of birth of the customer)
* bank_account_type (type of primary bank account)
* longitude_gps 
* latitude_gps
* bank_name_clients (name of the bank)
* bank_branch_clients (location of the branch - not compulsory - so missing in a lot of the cases)
* employment_status_clients (type of employment that customer has)
* level_of_education_clients (highest level of education)

### Performance data

* customerid (Primary key used to merge to other data)
* systemloanid (The id associated with the particular loan. The same customerId can have multiple systemloanid’s for each loan he/she has taken out)
* loannumber (The number of the loan that you have to predict)
* approveddate (Date that loan was approved)
* creationdate (Date that loan application was created)
* loanamount (Loan value taken)
* totaldue (Total repayment required to settle the loan - this is the capital loan value disbursed +interest and fees)
* termdays (Term of loan)
* referredby (customerId of the customer that referred this person - is missing, then not referred)
* good_bad_flag (good = settled loan on time; bad = did not settled loan on time) - this is the target variable that we need to predict

### Previous loans data 

* customerid (Primary key used to merge to other data)
* systemloanid (The id associated with the particular loan. The same customerId can have multiple systemloanid’s for each loan he/she has taken out)
* loannumber (The number of the loan that you have to predict)
* approveddate (Date that loan was approved)
* creationdate (Date that loan application was created)
* loanamount (Loan value taken)
* totaldue (Total repayment required to settle the loan - this is the capital loan value disbursed +interest and fees)
* closeddate (Date that the loan was settled)
* referredby (customerId of the customer that referred this person - is missing, then not referred)
* firstduedate (Date of first payment due in cases where the term is longer than 30 days. So in the case where the term is 60+ days - then there are multiple monthly payments due - and this dates reflects the date of the first payment)
* firstrepaiddate (Actual date that he/she paid the first payment as defined above)

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
traindemographics = pd.read_csv('traindemographics.csv')
trainperf = pd.read_csv('trainperf.csv', parse_dates= ['approveddate', 'creationdate'])
trainprevloans = pd.read_csv('trainprevloans.csv',parse_dates= ['approveddate', 'creationdate','closeddate'])

testdemographics = pd.read_csv('testdemographics.csv')
testperf = pd.read_csv('testperf.csv', parse_dates= ['approveddate', 'creationdate'])
testprevloans = pd.read_csv('testprevloans.csv',parse_dates= ['approveddate', 'creationdate','closeddate', 'firstduedate',
                                                              'firstrepaiddate'])

In [3]:
wdtrainperf = trainperf.drop(['approveddate', 'creationdate'], axis = 1)

In [4]:
wdtestperf = testperf.drop(['approveddate', 'creationdate'], axis = 1)

In [5]:
wdtrainperf['referredby'] = wdtrainperf['referredby'].fillna(0)

In [6]:
wdtestperf['referredby'] = wdtestperf['referredby'].fillna(0)

In [7]:
trainprevloans['referredby'] = trainprevloans['referredby'].fillna(0)

In [8]:
testprevloans['referredby'] = testprevloans['referredby'].fillna(0)

In [9]:
trainprevloans['creation_hour'] = trainprevloans['creationdate'].apply(lambda x: x.hour)
testprevloans['creation_hour'] = testprevloans['creationdate'].apply(lambda x: x.hour)

trainprevloans['approved_hour'] = trainprevloans['approveddate'].apply(lambda x: x.hour)
testprevloans['approved_hour'] = testprevloans['approveddate'].apply(lambda x: x.hour)

In [10]:
trainprevloans['creation_min'] = trainprevloans['creationdate'].apply(lambda x: x.minute)
testprevloans['creation_min'] = testprevloans['creationdate'].apply(lambda x: x.minute)

trainprevloans['approved_min'] = trainprevloans['approveddate'].apply(lambda x: x.minute)
testprevloans['approved_min'] = testprevloans['approveddate'].apply(lambda x: x.minute)

In [11]:
trainprevloans['aprroved_creation_time_diff'] = ((trainprevloans['approved_hour']
                                                 - trainprevloans['creation_hour'])*60
                                                 + (trainprevloans['approved_min']
                                                   - trainprevloans['creation_min']))

In [12]:
testprevloans['aprroved_creation_time_diff'] = ((testprevloans['approved_hour']
                                                 - testprevloans['creation_hour'])*60
                                                 + (testprevloans['approved_min']
                                                   - testprevloans['creation_min']))

In [13]:
trainprevloans['aprroved_creation_time_diff'] = np.abs(trainprevloans['aprroved_creation_time_diff'])

In [14]:
trainprevloans['approveddate'] = pd.to_datetime(trainprevloans['approveddate'])
trainprevloans['firstduedate'] = pd.to_datetime(trainprevloans['firstduedate'])
trainprevloans['firstrepaiddate'] = pd.to_datetime(trainprevloans['firstrepaiddate'])

In [15]:
testprevloans['approveddate'] = pd.to_datetime(testprevloans['approveddate'])
testprevloans['firstduedate'] = pd.to_datetime(testprevloans['firstduedate'])
testprevloans['firstrepaiddate'] = pd.to_datetime(testprevloans['firstrepaiddate'])

In [16]:
trainprevloans['closeddate'] = pd.to_datetime(trainprevloans['closeddate'])
testprevloans['closeddate'] = pd.to_datetime(testprevloans['closeddate'])

In [17]:
trainprevloans['closedapproved'] = (trainprevloans['closeddate'].apply(lambda x: x.month)
                                - trainprevloans['approveddate'].apply(lambda x: x.month)
                               ) * 30 + ((trainprevloans['closeddate'].apply(lambda x: x.day)
                                - (trainprevloans['approveddate'].apply(lambda x: x.day))))

In [18]:
testprevloans['closedapproved'] = (testprevloans['closeddate'].apply(lambda x: x.month)
                                - testprevloans['approveddate'].apply(lambda x: x.month)
                               ) * 30 + ((testprevloans['closeddate'].apply(lambda x: x.day)
                                - (testprevloans['approveddate'].apply(lambda x: x.day))))

In [19]:
trainprevloans['dueandpaid'] = (trainprevloans['firstrepaiddate'].apply(lambda x: x.month)
                                - trainprevloans['firstduedate'].apply(lambda x: x.month)
                               ) * 30 + ((trainprevloans['firstrepaiddate'].apply(lambda x: x.day)
                                - (trainprevloans['firstduedate'].apply(lambda x: x.day))))

In [20]:
testprevloans['dueandpaid'] = (testprevloans['firstrepaiddate'].apply(lambda x: x.month)
                                - testprevloans['firstduedate'].apply(lambda x: x.month)
                               ) * 30 + ((testprevloans['firstrepaiddate'].apply(lambda x: x.day)
                                - (testprevloans['firstduedate'].apply(lambda x: x.day))))

In [21]:
for i in trainprevloans.index:
    trainprevloans['Default'] = 1

In [22]:
for i in testprevloans.index:
    testprevloans['Default'] = 1

In [23]:
trainprevloans['fr_year'] = trainprevloans['firstrepaiddate'].apply(lambda x: x.year)
trainprevloans['fd_year'] = trainprevloans['firstduedate'].apply(lambda x: x.year)

testprevloans['fr_year'] = testprevloans['firstrepaiddate'].apply(lambda x: x.year)
testprevloans['fd_year'] = testprevloans['firstduedate'].apply(lambda x: x.year)

In [24]:
trainprevloans['fr_month'] = trainprevloans['firstrepaiddate'].apply(lambda x: x.month)
trainprevloans['fd_month'] = trainprevloans['firstduedate'].apply(lambda x: x.month)

testprevloans['fr_month'] = testprevloans['firstrepaiddate'].apply(lambda x: x.month)
testprevloans['fd_month'] = testprevloans['firstduedate'].apply(lambda x: x.month)

In [25]:
trainprevloans['fr_day'] = trainprevloans['firstrepaiddate'].apply(lambda x: x.day)
trainprevloans['fd_day'] = trainprevloans['firstduedate'].apply(lambda x: x.day)

testprevloans['fr_day'] = testprevloans['firstrepaiddate'].apply(lambda x: x.day)
testprevloans['fd_day'] = testprevloans['firstduedate'].apply(lambda x: x.day)

In [26]:
trainprevloans[(trainprevloans['fr_year'] > trainprevloans['fd_year']) | (trainprevloans['fr_month'] > trainprevloans['fd_month']) | (trainprevloans['fr_day'] > trainprevloans['fd_day'])].apply(len)

customerid                     5693
systemloanid                   5693
loannumber                     5693
approveddate                   5693
creationdate                   5693
loanamount                     5693
totaldue                       5693
termdays                       5693
closeddate                     5693
referredby                     5693
firstduedate                   5693
firstrepaiddate                5693
creation_hour                  5693
approved_hour                  5693
creation_min                   5693
approved_min                   5693
aprroved_creation_time_diff    5693
closedapproved                 5693
dueandpaid                     5693
Default                        5693
fr_year                        5693
fd_year                        5693
fr_month                       5693
fd_month                       5693
fr_day                         5693
fd_day                         5693
dtype: int64

In [27]:
def default(df):
    if (df['fr_year'] > df['fd_year']) | (df['fr_month'] > df['fd_month']) | (df['fr_day'] > df['fd_day']):
        df['Default'] = 0
        return df['Default']
    else:
        return df['Default']

In [28]:
func_check = trainprevloans.head()

In [29]:
func_check.apply(default, axis = 1)

0    1
1    1
2    0
3    1
4    0
dtype: int64

In [30]:
trainprevloans['Default'] = trainprevloans.apply(default, axis = 1)

In [31]:
testprevloans['Default'] = testprevloans.apply(default, axis = 1)

In [32]:
trainprevloans = trainprevloans.drop(['creationdate'], axis = 1)

In [33]:
testprevloans = testprevloans.drop(['creationdate'], axis = 1)

In [34]:
trainprevloans = trainprevloans.drop(['closeddate','approveddate',
                                     'firstrepaiddate', 'firstduedate'],
                                    axis = 1)

In [35]:
testprevloans = testprevloans.drop(['closeddate','approveddate',
                                     'firstrepaiddate', 'firstduedate'],
                                    axis = 1)

In [36]:
trainprevloans['interest'] = trainprevloans['totaldue'] - trainprevloans['loanamount']

In [37]:
trainperf['interests'] = trainperf['totaldue'] - trainperf['loanamount']

In [38]:
testprevloans['interest'] = testprevloans['totaldue'] - testprevloans['loanamount']

In [39]:
testperf['interests'] = testperf['totaldue'] - testperf['loanamount']

In [40]:
#trainprevloans = trainprevloans.drop(['totaldue','loanamount'], axis = 1)

In [41]:
#trainperf = trainperf.drop(['totaldue','loanamount'], axis = 1)
#testperf = testperf.drop(['totaldue','loanamount'], axis = 1)

In [42]:
#testprevloans = testprevloans.drop(['totaldue','loanamount'], axis = 1)

In [43]:
testprevloans.shape

(5907, 22)

In [44]:
tpandtlmerge = trainprevloans.merge(wdtrainperf, on = 'customerid', how = 'outer')

In [45]:
tspandtslmerge = testprevloans.merge(wdtestperf, on = 'customerid', how = 'outer')

In [46]:
tspandtslmerge.shape

(5915, 28)

In [47]:
tpandtlmerge['good_bad_flag'] = tpandtlmerge['good_bad_flag'].map({'Good': 1,'Bad' : 0})

In [48]:
tpandtlmerge.isnull().sum()

customerid                     0
systemloanid_x                 9
loannumber_x                   9
loanamount_x                   9
totaldue_x                     9
termdays_x                     9
referredby_x                   9
creation_hour                  9
approved_hour                  9
creation_min                   9
approved_min                   9
aprroved_creation_time_diff    9
closedapproved                 9
dueandpaid                     9
Default                        9
fr_year                        9
fd_year                        9
fr_month                       9
fd_month                       9
fr_day                         9
fd_day                         9
interest                       9
systemloanid_y                 0
loannumber_y                   0
loanamount_y                   0
totaldue_y                     0
termdays_y                     0
referredby_y                   0
good_bad_flag                  0
dtype: int64

In [49]:
tmerge = tpandtlmerge.groupby('customerid')[tpandtlmerge.columns].mean()

In [50]:
tsmerge = tspandtslmerge.groupby('customerid')[tspandtslmerge.columns].mean()

In [51]:
tmerge.reset_index(inplace= True)

In [52]:
tsmerge.reset_index(inplace=True)

In [53]:
X = tmerge.iloc[:, :-1]
y = tmerge['good_bad_flag']

In [54]:
tmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4368 entries, 0 to 4367
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   customerid                   4368 non-null   object 
 1   systemloanid_x               4359 non-null   float64
 2   loannumber_x                 4359 non-null   float64
 3   loanamount_x                 4359 non-null   float64
 4   totaldue_x                   4359 non-null   float64
 5   termdays_x                   4359 non-null   float64
 6   creation_hour                4359 non-null   float64
 7   approved_hour                4359 non-null   float64
 8   creation_min                 4359 non-null   float64
 9   approved_min                 4359 non-null   float64
 10  aprroved_creation_time_diff  4359 non-null   float64
 11  closedapproved               4359 non-null   float64
 12  dueandpaid                   4359 non-null   float64
 13  Default           

In [55]:
X = np.abs(X.drop('customerid', axis = 1))

In [56]:
tsmerge.drop('customerid', axis = 1, inplace= True)

In [57]:
X['interestx'] = X['totaldue_x'] - X['loanamount_x']

In [58]:
X['interesty'] = X['totaldue_y'] - X['loanamount_y']

In [59]:
tsmerge['interestx'] = tsmerge['totaldue_x'] - tsmerge['loanamount_x']

In [60]:
tsmerge['interesty'] = tsmerge['totaldue_y'] - tsmerge['loanamount_y']

In [61]:
X = X.drop(['totaldue_y','totaldue_x','loanamount_x','loanamount_y', 'systemloanid_y', 'systemloanid_x'], axis = 1)

In [62]:
tsmerge = tsmerge.drop(['totaldue_y','totaldue_x','loanamount_x','loanamount_y', 'systemloanid_y', 'systemloanid_x'], axis = 1)

In [63]:
tsmerge = abs(tsmerge)

In [64]:
X = X.fillna(np.round(X.mean()))

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
from sklearn.ensemble import RandomForestClassifier

In [100]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4,
                                                   random_state = 101)

In [101]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth = None, min_samples_split=10,
                            max_leaf_nodes = 20)

In [102]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=20, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [103]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [104]:
rfcpred = rfc.predict(X_test)

In [105]:
error = 1 - accuracy_score(y_test, rfcpred)
print(error)

0.20080091533180777


In [106]:
print(classification_report(y_test, rfcpred))

              precision    recall  f1-score   support

           0       0.62      0.09      0.16       364
           1       0.81      0.98      0.89      1384

    accuracy                           0.80      1748
   macro avg       0.71      0.54      0.52      1748
weighted avg       0.77      0.80      0.74      1748



In [107]:
print(confusion_matrix(y_test, rfcpred))

[[  34  330]
 [  21 1363]]


In [108]:
tsmerge = tsmerge.fillna(0)

In [109]:
rfcpred = rfc.predict(tsmerge)

In [110]:
ss = pd.read_csv('SampleSubmission (1).csv')
ss.head()

,customerid,Good_Bad_flag
0,8a28afc7474813a40147639ec637156b,1
1,8a3735d5518aba7301518ac34413010d,1
2,8a76e7d443e6e97c0143ed099d102b1d,1
3,8a818823525dceef01525deda2480384,1
4,8a818926522ea5ef01523aff15c37482,1


In [111]:
ss.Good_Bad_flag = rfcpred

In [112]:
ss.to_csv('SubmissonRFC.csv', index = False)

## Stacking

Specifically, we will evaluate the following five algorithms:

* Logistic Regression.
* k-Nearest Neighbors.
* Decision Tree.
* Support Vector Machine.
* Naive Bayes.